Description :

The hackathon hosted by Zindi platform:
 
Task :

The task is to build the Machine learning predictive Model hat determines the likelihood for a customer to churn - to seek an alternative insurer or simply stop paying for insurance altogether.

**Evaluation :**

The evaluation metrics of this challenge is  log_loss

**Modelling**

I will use ensemble Algorithms to create my model 
 
 Lightgm  and catboost Algorithms

In [1]:
!pip install catboost

     |████████████████████████████████| 64.8MB 43kB/s 


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#TQDM is a progress bar library with good support for nested loops and Jupyter/IPython notebooks.
from tqdm import tqdm
import math
import gc
#
from sklearn.metrics import log_loss,accuracy_score

##
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Data Collection

In [0]:
train = pd.read_csv("/content/drive/My Drive/Zindi_weekendz_hackathons/The Zimnat Insurance Assurance(Hackathon)/Data/train.csv")
policy = pd.read_csv("/content/drive/My Drive/Zindi_weekendz_hackathons/The Zimnat Insurance Assurance(Hackathon)/Data/policy_data.csv")
client = pd.read_csv("/content/drive/My Drive/Zindi_weekendz_hackathons/The Zimnat Insurance Assurance(Hackathon)/Data/client_data.csv")
payment = pd.read_csv("/content/drive/My Drive/Zindi_weekendz_hackathons/The Zimnat Insurance Assurance(Hackathon)/Data/payment_history.csv")
sub = pd.read_csv("/content/drive/My Drive/Zindi_weekendz_hackathons/The Zimnat Insurance Assurance(Hackathon)/Data/sample_sub.csv")

In [35]:
#preview top client data
client.head()

,Policy ID,NPH_TITLE,NPH_LASTNAME,NPH_SEX,NPH_BIRTHDATE,NAD_ADDRESS1,NAD_ADDRESS2
0,PID_P1UHZRT,Ms.,NaN,F,1997,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT
1,PID_GUI5GQJ,Mr.,NPH_LASTNAME_L256UFJ,M,1954,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT
2,PID_LECMECN,Mr.,NPH_LASTNAME_3YMQL4X,M,1923,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT
3,PID_BCSLJHG,Mrs.,NPH_LASTNAME_IYMYRSV,F,1973,NaN,NaN
4,PID_YA1G5R6,Ms.,NPH_LASTNAME_HJ3MUUR,F,1935,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT


In [36]:
#preview top payment data
payment.head()

,Policy ID,AMOUNTPAID,DATEPAID,POSTDATE,PREMIUMDUEDATE
0,PID_VTBGKIZ,1287.423813,2018-12-29 00:00:00,2018-12-29 00:00:00,2018-01-11 00:00:00
1,PID_9J9V5YQ,0.000000,2017-04-10 00:00:00,2017-04-10 17:30:00,2017-01-10 00:00:00
2,PID_RT2SI59,3435.950888,2018-12-24 00:00:00,2018-12-28 00:00:00,2019-01-01 00:00:00
3,PID_NCXO0DU,3196.364996,2018-06-26 00:00:00,2018-06-26 00:00:00,2018-01-07 00:00:00
4,PID_FZYCT7V,0.000000,2018-03-12 00:00:00,2018-03-12 17:09:07,2018-01-12 00:00:00


In [0]:
# splitting train & test sets
test_set =train.loc[train["Lapse"]=="?"]

train_set=train.loc[train["Lapse"] !="?"]

In [38]:
#check the shape of datasets
train_set.shape , test_set.shape

((7978, 3), (43707, 3))

Creating Train Data

In [39]:
#merging trainset with policy data, to create train dataframe
train_df          = pd.merge(policy, train_set, on="Policy ID",how='left')
train_df['Lapse'] = train_df['Lapse'].replace(np.nan,0)
train_df['Lapse'] = train_df['Lapse'].replace('1',1)
train_df.tail()

,Policy ID,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,NPH_LASTNAME,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY,Lapse,Lapse Year
282810,PID_7P0IXF8,1/5/2017,PPR_PRODCD_KOFUYNN,2109.812579,NPH_LASTNAME_C11UH3D,3,555,74968.903115,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_1OCF2N0,PCL_LOCATCODE_SPQHMX5,OCCUPATION_IKCIDKW,CATEGORY_R821UZV,0,NaN
282811,PID_Q1QLPUQ,1/11/2018,PPR_PRODCD_KOFUYNN,346.027849,NPH_LASTNAME_4PA8HYE,2,222,45513.429325,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_M4MM4QC,PCL_LOCATCODE_SPQHMX5,OCCUPATION_IKCIDKW,CATEGORY_R821UZV,0,NaN
282812,PID_NEDS3A9,1/7/2017,PPR_PRODCD_KOFUYNN,217.975890,NPH_LASTNAME_XF7JXJN,3,333,45513.429325,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_DAP1YRU,PCL_LOCATCODE_0T6GYGX,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,1,2018
282813,PID_U8QKEJK,1/6/2017,PPR_PRODCD_W0F6GK1,5233.874657,NPH_LASTNAME_TE9AFPP,2,222,238857.872515,NLO_TYPE_XAJI0Y6,1075.892081,AAG_AGCODE_L1YVKZO,PCL_LOCATCODE_PEU5TF2,OCCUPATION_IKCIDKW,CATEGORY_R821UZV,0,NaN
282814,PID_5US8QG7,1/9/2019,PPR_PRODCD_W0F6GK1,3990.141182,NPH_LASTNAME_3WBJCBF,1,111,393442.396414,NLO_TYPE_XAJI0Y6,820.226234,AAG_AGCODE_L1YVKZO,PCL_LOCATCODE_PEU5TF2,OCCUPATION_IKCIDKW,CATEGORY_R821UZV,0,NaN


In [40]:
#we have two classes on from train data lets count the occurance of each class
train_df['Lapse'].value_counts()

0    225260
1     57555
Name: Lapse, dtype: int64

In [41]:
#check the merged train shape
print('The shape of our train file will be ',train_df.shape)

The shape of our train file will be  (282815, 16)


Creating Test Data

In [42]:
# merging test_set with policy data, to create test dataframe
test_df  = pd.merge(test_set,policy, on="Policy ID",how='left')
test_df  = test_df.drop_duplicates(subset='Policy ID')
test_df.tail()

,Policy ID,Lapse,Lapse Year,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,NPH_LASTNAME,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY
225238,PID_FSR6DS3,?,?,1/6/2019,PPR_PRODCD_B2KVCE7,191.160414,NPH_LASTNAME_TU0EFBY,3,555,NaN,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_218G086,PCL_LOCATCODE_SPQHMX5,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB
225252,PID_98JLBWI,?,?,1/6/2019,PPR_PRODCD_OK3TM96,614.308232,NPH_LASTNAME_8B3JSO8,1,111,45513.429325,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_BKH543Z,PCL_LOCATCODE_0T6GYGX,OCCUPATION_Y41IBLJ,CATEGORY_GWW4FYB
225254,PID_HM8H99C,?,?,1/10/2017,PPR_PRODCD_64QNIHM,3187.154301,NPH_LASTNAME_XF7JXJN,1,111,86253.370871,NLO_TYPE_XAJI0Y6,537.201791,AAG_AGCODE_9X0IGLA,PCL_LOCATCODE_7VFS3EQ,OCCUPATION_WI64CIY,CATEGORY_GWW4FYB
225256,PID_KQH5U5I,?,?,1/8/2019,PPR_PRODCD_64QNIHM,2795.069380,NPH_LASTNAME_P6QHWZW,1,111,109268.927230,NLO_TYPE_XTHV3A3,609.054794,AAG_AGCODE_GSLG1SX,PCL_LOCATCODE_SPQHMX5,OCCUPATION_WI64CIY,CATEGORY_GWW4FYB
225258,PID_HWFS2BA,?,?,1/6/2018,PPR_PRODCD_64QNIHM,2795.069380,NPH_LASTNAME_W96CGOW,1,111,175645.703656,NLO_TYPE_XTHV3A3,609.054794,AAG_AGCODE_7T62ME4,PCL_LOCATCODE_PI2W0SA,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB


In [0]:
# save target values to target_var column
target_var = train_df['Lapse'].values

In [0]:
#Selecting some features
to_drop_cols = ['Policy ID','Lapse Year','NP2_EFFECTDATE','NPH_LASTNAME','Lapse','NLO_AMOUNT']
train_df   = train_df.drop(to_drop_cols,axis=1)
test_df  = test_df.drop(to_drop_cols,axis=1)

Treating null values with mean()

In [0]:
train_df['NPR_SUMASSURED'].fillna(train_df['NPR_SUMASSURED'].mean(), inplace=True)

In [0]:
test_df['NPR_SUMASSURED'].fillna(test_df['NPR_SUMASSURED'].mean(), inplace=True)

Dealing With Categorical Features

In [0]:
#filtering categorical data from train_df
cate_features_index = np.where(train_df.dtypes == object)[0]

In [0]:
#selecting dependence and independent features
x_train = train_df.values
x_test  = test_df.values
train_y = target_var

Modelling Part

In [0]:
from catboost import CatBoostClassifier

from sklearn.model_selection import StratifiedKFold

In [56]:
for i in range(1):
    print('iterations ',i)
    n_splits               = 30
    oof_pred               = np.zeros((len(x_train), ))
    skf                    = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    y_pred                 = np.zeros((len(x_test), ))
    
    ##########  Performing stratified k fold on the data 
    for fold, (tr_ind, val_ind) in enumerate(skf.split(x_train, train_y)):

        model = CatBoostClassifier(random_state=42,n_estimators=1000,nan_mode='Max',learning_rate=0.02,depth=4,reg_lambda=150,eval_metric='Logloss',use_best_model=True,silent=True)
    
        X_train, X_val     = x_train[tr_ind], x_train[val_ind]
        y_train, y_val     = train_y[tr_ind], train_y[val_ind]
        
        ##### Model fit on the k fold train data and its label
        model.fit(X_train,y_train,cat_features = cate_features_index,eval_set=(X_val,y_val))
        
        ##### Validation done on the validation fold
        val_pred = model.predict_proba(X_val)[:,1]


         ### Validation outcome determined to test the overall AUC of the model
        oof_pred[val_ind]  = val_pred
        
        print('validation logloss fold-',fold+1,': ',log_loss(y_val, val_pred))
        oof_pred[val_ind] = val_pred
        y_pred += model.predict_proba(x_test)[:,1] / (n_splits)
    y_pred_final = y_pred

iterations  0
validation logloss fold- 1 :  0.4048900730769501
validation logloss fold- 2 :  0.4020535186865175
validation logloss fold- 3 :  0.4021380043781539
validation logloss fold- 4 :  0.4030343680176922
validation logloss fold- 5 :  0.40134103997374165
validation logloss fold- 6 :  0.4024501843098005
validation logloss fold- 7 :  0.39377093865059504
validation logloss fold- 8 :  0.3979101079083925
validation logloss fold- 9 :  0.40373966238673
validation logloss fold- 10 :  0.395138452437403
validation logloss fold- 11 :  0.39786018485800895
validation logloss fold- 12 :  0.39979670611304075
validation logloss fold- 13 :  0.4008360781889119
validation logloss fold- 14 :  0.3924477721267053
validation logloss fold- 15 :  0.3959801752138979
validation logloss fold- 16 :  0.3984943855982512
validation logloss fold- 17 :  0.4038292954347767
validation logloss fold- 18 :  0.39892763758653293
validation logloss fold- 19 :  0.4046268497988171
validation logloss fold- 20 :  0.3997661384

In [0]:
submission_df = pd.DataFrame({'Policy ID':sub['Policy ID'],'Lapse':y_pred_final})
submission_df.to_csv('My_submission_file.csv',index=False)

In [58]:
submission_df.head()

,Policy ID,Lapse
0,PID_4928TWH,0.235037
1,PID_KBLLEGK,0.001011
2,PID_90F0QA3,0.002164
3,PID_18F3NHF,0.024884
4,PID_SX4QUVO,0.256847
